In [1]:
########################################
#             PATH SETUP
########################################

import sys
import glob
import os
sys.path.insert(0, os.path.abspath("."))

########################################
#             LIBRERIES SETUP
########################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from pathlib import Path
from collections.abc import Iterable
from typing import Dict
import seaborn as sns
import math, re
########################################
#             SIMULATION LIBRARIES
########################################
from lemer.rngs import MultiStreamRNG
from lemer.rvms import *
from typing import List, Optional, Tuple
from simulator.simulation import Simulation

In [ ]:

BASE_DIR = ".output_simulation"

# configurazione all'inizio del notebook
OBJ = 2   # scegli 1, 2 o 3

# costruisco dinamicamente il nome del file
config_file = f"obj{OBJ}.json"
config_path = f"analytic_sweep_lambda.csv"
TITLE_PREFIX = f"OBJ{OBJ}"


In [ ]:

def find_csv_for_config(config_path: str, base_dir=BASE_DIR, target="results") -> List[str]:
    cfg_name = Path(config_path).stem
    pattern  = os.path.join(base_dir, f"{target}_{cfg_name}*.csv")
    return sorted(glob.glob(pattern))




def _parse_lambda_from_filename(fname: str) -> Optional[float]:
    base = Path(fname).stem.lower()
    m = re.search(r"(?:lam|lambda|gamma|load)[=_]?([0-9]+(?:\.[0-9]+)?)", base)
    if m: return float(m.group(1))
    # fallback prudente
    for tok in re.findall(r"([0-9]+\.[0-9]+)", base):
        val = float(tok)
        if 0.1 <= val <= 5.0:
            return val
    return None

def load_runs(csv_files: List[str]) -> pd.DataFrame:
    dfs = []
    for i, f in enumerate(csv_files):
        df = pd.read_csv(f)
        df["source"]  = os.path.basename(f)
        df["replica"] = i
        if "arrival_rate" not in df.columns:
            df["arrival_rate"] = _parse_lambda_from_filename(f)
        dfs.append(df)
    if not dfs:
        raise FileNotFoundError("Nessun CSV.")
    df = pd.concat(dfs, ignore_index=True)

    if "mean_response_time" not in df.columns:
        for alt in ("response_time_mean", "rt_mean"):
            if alt in df.columns:
                df = df.rename(columns={alt: "mean_response_time"})
                break
    return df

In [ ]:
conv_files=find_csv_for_config(config_file,target="conv")


In [ ]:



# Trova il CSV analitico a partire dal path della config (come fai per i results)
def find_analytic_csv_for_config(config_path: str, base_dir=BASE_DIR) -> str:
    cfg_name = Path(config_path).stem
    candidates = []
    patterns = [
        os.path.join(base_dir, f"analytic_sweep_{cfg_name}*.csv"),
        os.path.join(base_dir, f"{cfg_name}_analytic*.csv"),
        os.path.join(base_dir, "analytic_sweep_lambda.csv"),
        os.path.join(Path(base_dir).parent, "analytic_sweep_lambda.csv"),
    ]
    for pat in patterns:
        candidates += glob.glob(pat)

    if not candidates:
        raise FileNotFoundError(
            f"Nessun CSV analitico trovato per '{cfg_name}'. "
            f"Cercati pattern: {patterns}"
        )

    candidates.sort(key=lambda p: (len(Path(p).name), p), reverse=True)
    return candidates[0]

# Carica l’analitico nel formato atteso dal plot (colonne: arrival_rate, mean_response_time, X_max opzionale)
def load_analytic_models_for_config(config_path: str, base_dir=BASE_DIR) -> pd.DataFrame:
    path = find_analytic_csv_for_config(config_path, base_dir=base_dir)
    df = pd.read_csv(path)

    # uniforma la colonna lambda → arrival_rate
    if "arrival_rate" not in df.columns and "lambda" in df.columns:
        df = df.rename(columns={"lambda": "arrival_rate"})

    if "arrival_rate" not in df.columns:
        raise ValueError("Il CSV analitico deve contenere almeno 'arrival_rate' o 'lambda'.")

    # ordina per sicurezza
    return df.sort_values("arrival_rate").reset_index(drop=True)


# uso
analytic_df = load_analytic_models_for_config(config_path, base_dir=BASE_DIR)


In [ ]:
# ---------- serie (usa la tua colonna 'time') ----------
def _series_rt_from_conv(df, scope, *, arrival_rate=None, t_max=None, downsample_every=1, ewma_span=7, align_zero=True):
    dd = df[(df["scope"]==scope) & (df["metric"]=="mean_response_time")].copy()
    if arrival_rate is not None and "arrival_rate" in dd.columns:
        dd = dd[dd["arrival_rate"].astype(float).eq(float(arrival_rate))]
    if dd.empty:
        return None, None

    dd = dd.sort_values("time")
    t = dd["time"].astype(float).to_numpy()
    y = dd["value"].astype(float).to_numpy()

    if align_zero:
        t = t - t.min()

    if t_max is not None:
        ok = t <= float(t_max)
        t, y = t[ok], y[ok]

    if downsample_every > 1 and len(y) > downsample_every:
        t = t[::downsample_every]; y = y[::downsample_every]

    if ewma_span and len(y) > 3:
        y = pd.Series(y).ewm(span=ewma_span, adjust=False).mean().to_numpy()

    return t, y

# ---------- plot in stile "paper" ----------
def plot_transient_rt_paper_style(
    dfs_conv_list,                      # lista di DataFrame conv (uno per seed)
    *,
    arrival_rate: float,
    scopes=("NODE_A","NODE_B","NODE_P","OVERALL"),
    t_max: float = 65.0,                # finestra mostrata (s)
    downsample_every: int = 1,          # 1 = nessun downsample
    ewma_span: int = 7,                 # smoothing leggero
    analytic_df: pd.DataFrame | None = None,
    suptitle: str = "Response time of transient state"
):
    # layout 2×2
    titles_map = {"NODE_A":"A", "NODE_B":"B", "NODE_P":"P", "OVERALL":"SYSTEM"}
    fig, axes = plt.subplots(2, 2, figsize=(12, 7), squeeze=False)
    axes = axes.ravel()

    for ax, scope in zip(axes, scopes):
        plotted = False
        for i, df in enumerate(dfs_conv_list):
            seed_label = "-"
            if "seed" in df.columns and not df["seed"].isna().all():
                s0 = str(df["seed"].dropna().iloc[0])
                seed_label = s0 if not s0.isdigit() else str(int(float(s0)))

            t, y = _series_rt_from_conv(
                df, scope,
                arrival_rate=arrival_rate,
                t_max=t_max,
                downsample_every=downsample_every,
                ewma_span=ewma_span,
                align_zero=True
            )
            if t is None: 
                continue

            markevery = max(1, len(t)//30)
            ax.plot(t, y, linewidth=1.8, marker="o", markersize=3.5, markevery=markevery,
                    label=f"seed = {seed_label}")
            plotted = True

        # linea analitica (rossa tratteggiata) se disponibile
        if analytic_df is not None:
            theor = _analytic_rt_for_scope(analytic_df, arrival_rate, scope)
            if theor is not None:
                ax.axhline(theor, linestyle="--", color="tab:red", linewidth=2,
                           label=(f"analytical (λ={arrival_rate}, RT={theor:.3f})"
                                  if scope=="OVERALL" else f"analytical (RT={theor:.3f})"))
        ax.set_xticks(np.arange(0,25,step=6))
        ax.set_title(titles_map.get(scope, scope))
      
        ax.set_xlim(0, 48)
        ax.set_xlabel("Time (h)")
        ax.set_ylabel("Mean response time (s)")
        
        ax.grid(True, alpha=0.3)
        if plotted:
            ax.legend(fontsize=9)
        else:
            ax.text(0.5, 0.5, "nessun dato", ha="center", va="center", transform=ax.transAxes)

    fig.suptitle(suptitle)
    plt.tight_layout()
    plt.savefig("24_simple_obj1",dpi=300)
    plt.show()

In [ ]:
dfs=[]
for df in dfs_conv:
    dc=df.copy()
    dc["time"]=dc["time"].astype(float)/3600.0
    dfs.append(dc)


plot_transient_rt_paper_style(
    dfs_conv_list=dfs,
    arrival_rate=1.2,
    scopes=("OVERALL", "NODE_A","NODE_B","NODE_P"),  # TL=SYSTEM, TR=A, BL=P, BR=B
    t_max=(24),
    downsample_every=1,
    ewma_span=7,
    analytic_df=analytic_df
)